<a href="https://colab.research.google.com/github/daddyboy77/A.I-testing/blob/master/cartoonize_b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Setup
!pip install imageio==2.4.1
import os
repo = "CartoonGan-tensorflow"
!git clone https://github.com/mnicnc404/CartoonGan-tensorflow.git
os.chdir(os.path.join(repo))

from IPython.display import clear_output, display, Image
!pip3 install tensorflow-gpu==2.2.0-alpha0
!git clone https://www.github.com/keras-team/keras-contrib.git \
    && cd keras-contrib \
    && python convert_to_tf_keras.py \
    && USE_TF_KERAS=1 python setup.py install

!pip3 install matlab
!pip3 install moviepy
clear_output()
import tensorflow as tf
print(tf.__version__)

2.8.2


In [ ]:
#@title Link drive
from google.colab import drive
drive.mount('/content/drive')
clear_output()

In [ ]:
!pip install imageio==2.4.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.3 MB 7.7 MB/s 
  Created wheel for imageio: filename=imageio-2.4.1-py3-none-any.whl size=3303885 sha256=e062448e68ac87f5cdf4e6704809b66fd303137ef3d50429cb66fe4ca490ee40
  Stored in directory: /root/.cache/pip/wheels/46/20/07/7bb9c8c44e6ec2efa60fd0e6280094f53f65f41767ef69a5ee
Successfully built imageio
  Attempting uninstall: imageio
    Found existing installation: imageio 2.9.0
    Uninstalling imageio-2.9.0:
      Successfully uninstalled imageio-2.9.0


In [ ]:
#@title Process video
vdirectory = "/content/drive/MyDrive/Real/Alt.mp4" #@param {type: "string"}
vidname = vdirectory.split("/")[-1]
name = '.'.join(vidname.split(".")[:-1])
print("name: " + name)
print("vidname: " + vidname)
print("Test:" + "/content/" + vidname + "/")
 
#@title Analyze
import cv2
def count_frames(path, override=False):
    # grab a pointer to the video file and initialize the total
    # number of frames read
    video = cv2.VideoCapture(path)
    total = 0
    # if the override flag is passed in, revert to the manual
    # method of counting frames
    if override:
        total = count_frames_manual(video)
    # otherwise, let's try the fast way first
    else:
        # lets try to determine the number of frames in a video
        # via video properties; this method can be very buggy
        # and might throw an error based on your OpenCV version
        # or may fail entirely based on your which video codecs
        # you have installed
        try:
            # check if we are using OpenCV 3
            # otherwise, we are using OpenCV 2.4
            total = int(video.get(cv2.cv.CV_CAP_PROP_FRAME_COUNT))
        # uh-oh, we got an error -- revert to counting manually
        except:
            total = count_frames_manual(video)
    # release the video file pointer
    video.release()
    # return the total number of frames in the video
    return total
def count_frames_manual(video):
    # initialize the total number of frames read
    total = 0
    # loop over the frames of the video
    while True:
        # grab the current frame
        (grabbed, frame) = video.read()
     
        # check to see if we have reached the end of the
        # video
        if not grabbed:
            break
        # increment the total number of frames read
        total += 1
    # return the total number of frames in the video file
    return total
if __name__ == '__main__' :
    video = cv2.VideoCapture(vdirectory);
    # Find OpenCV version
    (major_ver, minor_ver, subminor_ver) = (cv2.__version__).split('.')
    if int(major_ver)  < 3 :
        fps = video.get(cv2.cv.CV_CAP_PROP_FPS)
        print ("Frames per second using video.get(cv2.cv.CV_CAP_PROP_FPS): {0}".format(fps))
    else :
        fps = video.get(cv2.CAP_PROP_FPS)
        print ("Frames per second using video.get(cv2.CAP_PROP_FPS) : {0}".format(fps))
    video.release()
print(fps)
print(count_frames(vdirectory))


from concurrent.futures import ProcessPoolExecutor, as_completed
import cv2
import multiprocessing
import os
import sys
 
 
def print_progress(iteration, total, prefix='', suffix='', decimals=3, bar_length=100):
    """
    Call in a loop to create standard out progress bar
    :param iteration: current iteration
    :param total: total iterations
    :param prefix: prefix string
    :param suffix: suffix string
    :param decimals: positive number of decimals in percent complete
    :param bar_length: character length of bar
    :return: None
    """
 
    format_str = "{0:." + str(decimals) + "f}"  # format the % done number string
    percents = format_str.format(100 * (iteration / float(total)))  # calculate the % done
    filled_length = int(round(bar_length * iteration / float(total)))  # calculate the filled bar length
    bar = '#' * filled_length + '-' * (bar_length - filled_length)  # generate the bar string
    sys.stdout.write('\r%s |%s| %s%s %s' % (prefix, bar, percents, '%', suffix)),  # write out the bar
    sys.stdout.flush()  # flush to stdout
 
 
def extract_frames(video_path, frames_dir, overwrite=False, start=-1, end=-1, every=1):
    """
    Extract frames from a video using OpenCVs VideoCapture
    :param video_path: path of the video
    :param frames_dir: the directory to save the frames
    :param overwrite: to overwrite frames that already exist?
    :param start: start frame
    :param end: end frame
    :param every: frame spacing
    :return: count of images saved
    """
 
    video_path = os.path.normpath(video_path)  # make the paths OS (Windows) compatible
    frames_dir = os.path.normpath(frames_dir)  # make the paths OS (Windows) compatible
 
    video_dir, video_filename = os.path.split(video_path)  # get the video path and filename from the path
 
    assert os.path.exists(video_path)  # assert the video file exists
 
    capture = cv2.VideoCapture(video_path)  # open the video using OpenCV
 
    if start < 0:  # if start isn't specified lets assume 0
        start = 0
    if end < 0:  # if end isn't specified assume the end of the video
        end = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))
 
    capture.set(1, start)  # set the starting frame of the capture
    frame = start  # keep track of which frame we are up to, starting from start
    while_safety = 0  # a safety counter to ensure we don't enter an infinite while loop (hopefully we won't need it)
    saved_count = 0  # a count of how many frames we have saved
 
    while frame < end:  # lets loop through the frames until the end
 
        _, image = capture.read()  # read an image from the capture
 
        if while_safety > 500:  # break the while if our safety maxs out at 500
            break
 
        # sometimes OpenCV reads None's during a video, in which case we want to just skip
        if image is None:  # if we get a bad return flag or the image we read is None, lets not save
            while_safety += 1  # add 1 to our while safety, since we skip before incrementing our frame variable
            continue  # skip
 
        if frame % every == 0:  # if this is a frame we want to write out based on the 'every' argument
            while_safety = 0  # reset the safety count
            save_path = os.path.join(frames_dir, video_filename, "{:010d}.jpg".format(frame))  # create the save path
            if not os.path.exists(save_path) or overwrite:  # if it doesn't exist or we want to overwrite anyways
                cv2.imwrite(save_path, image)  # save the extracted image
                saved_count += 1  # increment our counter by one
 
        frame += 1  # increment our frame count
 
    capture.release()  # after the while has finished close the capture
 
    return saved_count  # and return the count of the images we saved
 
 
def video_to_frames(video_path, frames_dir, overwrite=False, every=1, chunk_size=1000):
    """
    Extracts the frames from a video using multiprocessing
    :param video_path: path to the video
    :param frames_dir: directory to save the frames
    :param overwrite: overwrite frames if they exist?
    :param every: extract every this many frames
    :param chunk_size: how many frames to split into chunks (one chunk per cpu core process)
    :return: path to the directory where the frames were saved, or None if fails
    """
 
    video_path = os.path.normpath(video_path)  # make the paths OS (Windows) compatible
    frames_dir = os.path.normpath(frames_dir)  # make the paths OS (Windows) compatible
 
    video_dir, video_filename = os.path.split(video_path)  # get the video path and filename from the path
 
    # make directory to save frames, its a sub dir in the frames_dir with the video name
    os.makedirs(os.path.join(frames_dir, video_filename), exist_ok=True)
 
    capture = cv2.VideoCapture(video_path)  # load the video
    total = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))  # get its total frame count
    capture.release()  # release the capture straight away
 
    if total < 1:  # if video has no frames, might be and opencv error
        print("Video has no frames. Check your OpenCV + ffmpeg installation, can't read videos!!!\n"
              "You may need to install OpenCV by source not pip")
        return None  # return None
 
    frame_chunks = [[i, i+chunk_size] for i in range(0, total, chunk_size)]  # split the frames into chunk lists
    frame_chunks[-1][-1] = min(frame_chunks[-1][-1], total-1)  # make sure last chunk has correct end frame
 
    prefix_str = "Extracting frames from {}".format(video_filename)  # a prefix string to be printed in progress bar
 
    # execute across multiple cpu cores to speed up processing, get the count automatically
    with ProcessPoolExecutor(max_workers=multiprocessing.cpu_count()) as executor:
 
        futures = [executor.submit(extract_frames, video_path, frames_dir, overwrite, f[0], f[1], every)
                   for f in frame_chunks]  # submit the processes: extract_frames(...)
 
        for i, f in enumerate(as_completed(futures)):  # as each process completes
            print_progress(i, len(frame_chunks)-1, prefix=prefix_str, suffix='Complete')  # print it's progress
 
    return os.path.join(frames_dir, video_filename)  # when done return the directory containing the frames
 
Chunk_Slider = 25 #@param {type:"slider", min:0, max:1000, step:25}
Comparison = False #@param {type:"boolean"}
if __name__ == '__main__':
    # test it
    video_to_frames(video_path=vdirectory, frames_dir="/content/", overwrite=False, every=1, chunk_size=Chunk_Slider)
from moviepy.editor import *
if not os.path.exists("/content/sound"):
 os.mkdir("/content/sound")
video = VideoFileClip(vdirectory) 
audio = video.audio 
audioname = "/content/sound/" + name + ".mp3"
audio.write_audiofile(audioname)
#lear_output()

name: Alt
vidname: Alt.mp4
Test:/content/Alt.mp4/
Frames per second using video.get(cv2.CAP_PROP_FPS) : 29.615574458810634
29.615574458810634
154
Extracting frames from Alt.mp4 |####################################################################################################| 100.000% Complete[MoviePy] Writing audio in /content/sound/Alt.mp3


100%|██████████| 115/115 [00:00<00:00, 1179.85it/s]

[MoviePy] Done.


In [ ]:
!zip -r /content/paprikalt.zip "/content/paprika"

  adding: content/paprika/ (stored 0%)
  adding: content/paprika/0000000052.jpg (deflated 2%)
  adding: content/paprika/0000000087.jpg (deflated 2%)
  adding: content/paprika/0000000080.jpg (deflated 2%)
  adding: content/paprika/0000000134.jpg (deflated 2%)
  adding: content/paprika/0000000094.jpg (deflated 2%)
  adding: content/paprika/0000000152.jpg (deflated 2%)
  adding: content/paprika/0000000120.jpg (deflated 2%)
  adding: content/paprika/0000000007.jpg (deflated 2%)
  adding: content/paprika/0000000110.jpg (deflated 2%)
  adding: content/paprika/0000000069.jpg (deflated 2%)
  adding: content/paprika/0000000150.jpg (deflated 2%)
  adding: content/paprika/0000000001.jpg (deflated 2%)
  adding: content/paprika/0000000088.jpg (deflated 2%)
  adding: content/paprika/0000000115.jpg (deflated 2%)
  adding: content/paprika/0000000084.jpg (deflated 2%)
  adding: content/paprika/0000000048.jpg (deflated 2%)
  adding: content/paprika/0000000143.jpg (deflated 2%)
  adding: content/paprika/

In [ ]:
from google.colab import files
files.download("/content/paprikalt.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
Comparison = False #@param {type:"boolean"}

In [ ]:
#@title Help
!python3 cartoonize.py --help

In [ ]:
import math
styles = "paprika" #@param ["shinkai", "hayao", "hosoda", "paprika"]
Inputdir = "/content/" + vidname
sn = "/content/" + styles
rfpsl = math.floor(fps)

In [ ]:
#@title Cartoonize retain(slow)
!rm -rf {sn}
!rm -rf "/content/comparison"
small = False
if(Comparison == True):
  !python cartoonize.py \
      --styles {styles} \
      --batch_size {rfpsl} \
      --comparison_view horizontal \
      --input_dir {Inputdir} \
      --keep_original_size \
      --output_dir "/content/"
else:
    !python cartoonize.py \
      --styles {styles} \
      --batch_size {rfpsl} \
      --input_dir {Inputdir} \
      --keep_original_size \
      --output_dir "/content/"
#clear_output()

[2022-03-30 17:35:50] [Cartoonizer] [INFO] Transformed images will be saved to `/content/` folder.
[2022-03-30 17:35:53] [Cartoonizer] [INFO] Cartoonizing images using paprika style...
[2022-03-30 17:35:53] [Cartoonizer] [INFO] Preparing to transform 281 images from `/content/P.mp4` directory...
Transforming: 100% 281/281 [03:12<00:00,  1.46it/s, File=0000000167.jpg]
[2022-03-30 17:39:06] [Cartoonizer] [INFO] Total processing time: 0:03:16.500108


In [ ]:
#@title Cartoonize resize
!rm -rf {sn}
small = True
!rm -rf "/content/comparison"
if(Comparison == True):
  !python3 cartoonize.py \
      --styles {styles} \
      --batch_size {rfpsl} \
      --comparison_view horizontal \
      --input_dir {Inputdir} \
      --output_dir "/content/"
else:
    !python3 cartoonize.py \
      --styles {styles} \
      --batch_size {rfpsl} \
      --input_dir {Inputdir} \
      --output_dir "/content/"

[2022-10-07 12:30:49] [Cartoonizer] [INFO] Transformed images will be saved to `/content/` folder.
[2022-10-07 12:30:52] [Cartoonizer] [INFO] Cartoonizing images using paprika style...
[2022-10-07 12:30:52] [Cartoonizer] [INFO] Preparing to transform 153 images from `/content/Alt.mp4` directory...
Transforming: 100% 153/153 [00:24<00:00,  6.19it/s, File=0000000090.jpg]
[2022-10-07 12:31:17] [Cartoonizer] [INFO] Total processing time: 0:00:28.174785


In [ ]:
#@title Compile
import cv2
import numpy as np
import os
import random 
from os.path import isfile, join
import shutil
 
numberList = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41]
entil = random.choice(numberList)

if(small == True):
  add = " rough"
else:
  add = " synced"
def convert_frames_to_video(pathIn,pathOut,fps):
    frame_array = []
    files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]
 
    #for sorting the file names properly
    files.sort(key = lambda x: int(x[5:-4]))
 
    for i in range(len(files)):
        filename=pathIn + files[i]
        #reading each files
        img = cv2.imread(filename)
        #print(img)
        height, width, layers = img.shape
        size = (width,height)
        print(filename)
        #inserting the frames into an image array
        frame_array.append(img)
 
    out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
 
    for i in range(len(frame_array)):
        # writing to a image array
        out.write(frame_array[i])
    out.release()
def main():
    pathIn= "/content/" + styles + "/"
    pathOut = "/content/drive/MyDrive/Cartoonized/" + name + "tmp" + styles + add + ".mp4"
    convert_frames_to_video(pathIn, pathOut, fps)

if __name__=="__main__":
    main()
pathIn= "/content/" + styles + "/"
pathOut = "/content/drive/MyDrive/Cartoonized/" + name + "tmp" + styles + add + ".mp4"
npathOut = "/content/drive/MyDrive/Cartoonized/" + name + " " + styles + add + ".mp4"


patht = "/content/drive/MyDrive/Cartoonized/comparison/" + name + "tmp" + styles + add + " comparison" + ".mp4"
npatht = "/content/drive/MyDrive/Cartoonized/comparison/" + name + " " + styles + add + " comparison" + ".mp4"
if(Comparison == True):
   convert_frames_to_video("/content/comparison/", patht, fps)
from moviepy.editor import *


videoclip = VideoFileClip(pathOut)
audioclip = AudioFileClip(audioname)
new_audioclip = CompositeAudioClip([audioclip])
videoclip.audio = new_audioclip
videoclip.write_videofile(npathOut)


delete_filepath = pathOut
open(delete_filepath, 'w').close() #overwrite and make the file blank instead - ref: https://stackoverflow.com/a/4914288/3553367
os.remove(delete_filepath)

if(Comparison == True):
  videoclipe = VideoFileClip(patht)
  audioclipe = AudioFileClip(audioname)
  new_audioclipe = CompositeAudioClip([audioclipe])
  videoclipe.audio = new_audioclipe
  videoclipe.write_videofile(npatht)
  delete_filepathd = patht
  open(delete_filepathd, 'w').close() 
  os.remove(delete_filepathd)



/content/paprika/0000000000.jpg
/content/paprika/0000000001.jpg
/content/paprika/0000000002.jpg
/content/paprika/0000000003.jpg
/content/paprika/0000000004.jpg
/content/paprika/0000000005.jpg
/content/paprika/0000000006.jpg
/content/paprika/0000000007.jpg
/content/paprika/0000000008.jpg
/content/paprika/0000000009.jpg
/content/paprika/0000000010.jpg
/content/paprika/0000000011.jpg
/content/paprika/0000000012.jpg
/content/paprika/0000000013.jpg
/content/paprika/0000000014.jpg
/content/paprika/0000000015.jpg
/content/paprika/0000000016.jpg
/content/paprika/0000000017.jpg
/content/paprika/0000000018.jpg
/content/paprika/0000000019.jpg
/content/paprika/0000000020.jpg
/content/paprika/0000000021.jpg
/content/paprika/0000000022.jpg
/content/paprika/0000000023.jpg
/content/paprika/0000000024.jpg
/content/paprika/0000000025.jpg
/content/paprika/0000000026.jpg
/content/paprika/0000000027.jpg
/content/paprika/0000000028.jpg
/content/paprika/0000000029.jpg
/content/paprika/0000000030.jpg
/content

OSError: ignored

In [ ]:
#@title Clear for new
!rm -rf "/content/paprika"
!rm -rf "/content/hosoda"
!rm -rf "/content/hayao"
!rm -rf "/content/shinkai"
!rm -rf "/content/comparison"
!rm -rf {Inputdir}
clear_output()

In [ ]:
#@title Ignore
#os.mkdir("/content/In/")
input_image_dir = "/content/"
output_image_dir = "/content/"

# import time
# if image_url:
#     img_filename = image_url.split("/")[-1]
#     name, ext = '.'.join(img_filename.split(".")[:-1]), img_filename.split(".")[-1]
#     new_name = '_'.join((name, str(int(time.time()))))
#     new_img_filename = '.'.join((new_name, ext))
#     image_path = os.path.join(input_image_dir, new_img_filename)
    
#     !wget {image_url} \
#         && mv {img_filename} {new_img_filename} \
#         && mv {new_img_filename} {image_path}

# Other s

In [ ]:
#@title Other one for extracting frames
#import cv2
vidcap = cv2.VideoCapture('video.mp4')
def getFrame(sec):
    vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
    hasFrames,image = vidcap.read()
    if hasFrames:
        cv2.imwrite("image"+str(count)+".jpg", image)     # save frame as JPG file
    return hasFrames
sec = 0
frameRate = fps #//it will capture image in each 0.5 second
count=1
success = getFrame(sec)
while success:
    count = count + 1
    sec = sec + frameRate
    sec = round(sec, 2)
    success = getFrame(sec)

In [ ]:
#@title For GIF
# a trick to show gif in notebook
if ".gif" in new_img_filename:
    png_path = new_img_filename + '.png'
    !cp {image_path} {png_path}

In [ ]:
#@title
!zip -r /content/SandyDancePaprika.zip /content/paprika/
clear_output()

In [ ]:
#@title Display
display(Image(png_path))

In [ ]:
#@title
if img_filename:
    if ".gif" in img_filename:
        generated_gif = os.path.join(output_image_dir, "comparison", new_img_filename)
        result_path = generated_gif + '.png'
        !cp {generated_gif} {result_path}
    else:
        result_path = os.path.join(output_image_dir, "comparison", new_img_filename)

In [ ]:
#@title
!tree {output_image_dir}

In [ ]:
!apt-get install tree
clear_output()

In [ ]:
import glob
os.listdir()
vditory = "/content/drive/My Drive/Datasets/Ynot/Fight.mp4" #@param {type: "string"}
vime = vditory.split("/")[-1]
nae = '.'.join(vime.split(".")[:-1])

['.config', 'Untitled Folder', 'sample_data']

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

# Modify

In [ ]:

import shutil

import os

pathrife = '/content/mk/' #@param {type: "string"}

isExist = os.path.exists(pathrife)

if not isExist:
  
  os.makedirs(pathrife)
  print(pathrife + " is created!")
shutil.copy(npathOut, pathrife)
print('/content/mk/' + )

P paprika synced is created!


'P paprika synced/P paprika synced.mp4'

In [ ]:
#@title
print(npathOut)

/content/drive/My Drive/Datasets/Cartoonized/P paprika synced.mp4


In [ ]:
#@title
%cd /content/ 
!git clone https://github.com/hzwer/arXiv2020-RIFE
!mkdir /content/arXiv2020-RIFE/train_log
%cd /content/arXiv2020-RIFE/train_log
!gdown --id 1APIzVeI-4ZZCEuIRE1m6WYfSCaOsi_7_
!7z e RIFE_trained_model_v3.6.zip
%cd /content/arXiv2020-RIFE/
!gdown --id 1i3xlKb7ax7Y70khcTcuePi6E7crO_dFc
!pip3 install -r requirements.txt
clear_output()

Please upload your video to content/arXiv2020-RIFE/video.mp4, or use our demo video.

In [ ]:
print(pathrife)
!python3 inference_video.py --exp=2 --video='/content/mk/P paprika synced.mp4' --output='/content/drive/MyDrive/Datasets/ScaledCart/'

Our demo.mp4 is 25FPS. You can adjust the parameters for your own perference.
For example: 
--fps=60 --exp=1 --video=mydemo.avi --png

In [ ]:
from IPython.display import display, Image
import moviepy.editor as mpy
display(mpy.ipython_display('/content/mk/20211227_191155 paprika synced_4X_120fps.mp4', height=256, max_duration=100.))

In [ ]:
!python3 inference_img.py --img demo/I0_0.png demo/I0_1.png
ffmpeg -r 10 -f image2 -i output/img%d.png -s 448x256 -vf "split[s0][s1];[s0]palettegen=stats_mode=single[p];[s1][p]paletteuse=new=1" output/slomo.gif
# Image interpolation